# **Comprensión de los datos**

Primero instalamos la librería de pandas-profiling para tener la última versión de esta librería y el funcionamiento sea correcto.

In [ ]:
pip install https://github.com/ydataai/pandas-profiling/archive/master.zip

Importamos todas las librerías que usaremos a lo largo de este notobook para la comprensión de los datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from matplotlib import style
import matplotlib.ticker as ticker
import pandas_profiling
import time

## Recolección de los datos

Partimos de una base de datos inicial creada en local mediante web scrapping en formato.csv, por lo que cargamos la base de datos desde la carpeta donde se encuentra el .csv en este notebook de Kaggle y la mostramos para tener una primera vista inicial de la base de datos. Todo esto lo hacemos con el uso de la librería pandas, para almacenar los datos en un dataset de pandas y que su acceso y su uso sea idóneo para el proyecto.

In [ ]:
bbdd=pd.read_csv("../input/bbddtfginicio/BaseDeDatosTFG.csv",sep=";",encoding='ISO-8859-1')
bbdd

Reemplazamos un carácter que no se ha cargado correctamente de la variable *Marcador* por el símbolo "-" para que su formato sea el formato común para este tipo de variables.

In [ ]:
for i in range(0,len(bbdd["Marcador"])):
    bbdd.loc[i,'Marcador']=bbdd.loc[i,'Marcador'].replace("\x96","-")
    bbdd.loc[i,'Posesión equipo local']=float(int(bbdd.loc[i,'Posesión equipo local'][:len(bbdd.loc[i,'Posesión equipo local'])-1])/100)
    bbdd.loc[i,'Posesión equipo visitante']=float(int(bbdd.loc[i,'Posesión equipo visitante'][:len(bbdd.loc[i,'Posesión equipo visitante'])-1])/100)

Rellenamos la columna de *Puntos totales local* y *Puntos totales visitante* a partir de los datos de las columnas *Puntos equipo local* y *Puntos equipo visitante* realizando una suma de los puntos que tienen en la jornada anterior más los puntos conseguidos en la jornada anterior teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'Puntos totales local'])) and (np.isnan(bbdd.loc[i,'Puntos totales visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Puntos totales local']=bbdd.at[j,'Puntos totales local']+bbdd.at[j,'Puntos equipo local']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Puntos totales local']=bbdd.at[j,'Puntos totales visitante']+bbdd.at[j,'Puntos equipo visitante']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Puntos totales visitante']=bbdd.at[j,'Puntos totales local']+bbdd.at[j,'Puntos equipo local']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Puntos totales visitante']=bbdd.at[j,'Puntos totales visitante']+bbdd.at[j,'Puntos equipo visitante']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'Puntos totales local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Puntos totales local']=bbdd.at[x,'Puntos totales local']+bbdd.at[x,'Puntos equipo local']
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Puntos totales local']=bbdd.at[x,'Puntos totales visitante']+bbdd.at[x,'Puntos equipo visitante']
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'Puntos totales visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Puntos totales visitante']=bbdd.at[y,'Puntos totales local']+bbdd.at[y,'Puntos equipo local']
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Puntos totales visitante']=bbdd.at[y,'Puntos totales visitante']+bbdd.at[y,'Puntos equipo visitante']
                break
            y=y-1

Rellenamos la columna de *PG local*, *PG visitante*, *PE local*, *PE visitante*, *PP local* y *PP visitante* a partir de los datos de las columnas *Puntos equipo local* y *Puntos equipo visitante* viendo los puntos conseguidos por cada equipo en la jornada anterior y viendo si han obtenido una victoria (3 puntos), un empate (1 punto) o una derrota (0 puntos), teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'PG local'])) and (np.isnan(bbdd.loc[i,'PG visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Puntos equipo local'] == 3:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG local']+1
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE local']
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP local']
                elif bbdd.at[j,'Puntos equipo local'] == 1:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG local']
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE local']+1
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP local']
                else:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG local']
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE local']
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP local']+1
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG visitante']+1
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE visitante']
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP visitante']
                elif bbdd.at[j,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG visitante']
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE visitante']+1
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP visitante']
                else:
                    bbdd.at[i,'PG local']=bbdd.at[j,'PG visitante']
                    bbdd.at[i,'PE local']=bbdd.at[j,'PE visitante']
                    bbdd.at[i,'PP local']=bbdd.at[j,'PP visitante']+1
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Puntos equipo local'] == 3:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG local']+1
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE local']
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP local']
                elif bbdd.at[j,'Puntos equipo local'] == 1:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG local']
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE local']+1
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP local']
                else:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG local']
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE local']
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP local']+1
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG visitante']+1
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE visitante']
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP visitante']
                elif bbdd.at[j,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG visitante']
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE visitante']+1
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP visitante']
                else:
                    bbdd.at[i,'PG visitante']=bbdd.at[j,'PG visitante']
                    bbdd.at[i,'PE visitante']=bbdd.at[j,'PE visitante']
                    bbdd.at[i,'PP visitante']=bbdd.at[j,'PP visitante']+1
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'PG local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                if bbdd.at[x,'Puntos equipo local'] == 3:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG local']+1
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE local']
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP local']
                elif bbdd.at[x,'Puntos equipo local'] == 1:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG local']
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE local']+1
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP local']
                else:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG local']
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE local']
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP local']+1
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                if bbdd.at[x,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG visitante']+1
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE visitante']
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP visitante']
                elif bbdd.at[x,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG visitante']
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE visitante']+1
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP visitante']
                else:
                    bbdd.at[i,'PG local']=bbdd.at[x,'PG visitante']
                    bbdd.at[i,'PE local']=bbdd.at[x,'PE visitante']
                    bbdd.at[i,'PP local']=bbdd.at[x,'PP visitante']+1
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'PG visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[y,'Puntos equipo local'] == 3:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG local']+1
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE local']
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP local']
                elif bbdd.at[y,'Puntos equipo local'] == 1:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG local']
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE local']+1
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP local']
                else:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG local']
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE local']
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP local']+1
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[y,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG visitante']+1
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE visitante']
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP visitante']
                elif bbdd.at[y,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG visitante']
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE visitante']+1
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP visitante']
                else:
                    bbdd.at[i,'PG visitante']=bbdd.at[y,'PG visitante']
                    bbdd.at[i,'PE visitante']=bbdd.at[y,'PE visitante']
                    bbdd.at[i,'PP visitante']=bbdd.at[y,'PP visitante']+1
                break
            y=y-1

Rellenamos la columna de *Goles a favor local* y *Goles a favor visitante* a partir de los datos de las columnas *Goles equipo local* y *Goles equipo visitante* realizando una suma de los goles a favor que tienen en la jornada anterior más los goles a favor conseguidos en la jornada anterior, teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'Goles a favor local'])) and (np.isnan(bbdd.loc[i,'Goles a favor visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Goles a favor local']=bbdd.at[j,'Goles a favor local']+bbdd.at[j,'Goles equipo local']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Goles a favor local']=bbdd.at[j,'Goles a favor visitante']+bbdd.at[j,'Goles equipo visitante']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Goles a favor visitante']=bbdd.at[j,'Goles a favor local']+bbdd.at[j,'Goles equipo local']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Goles a favor visitante']=bbdd.at[j,'Goles a favor visitante']+bbdd.at[j,'Goles equipo visitante']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'Goles a favor local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Goles a favor local']=bbdd.at[x,'Goles a favor local']+bbdd.at[x,'Goles equipo local']
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Goles a favor local']=bbdd.at[x,'Goles a favor visitante']+bbdd.at[x,'Goles equipo visitante']
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'Goles a favor visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Goles a favor visitante']=bbdd.at[y,'Goles a favor local']+bbdd.at[y,'Goles equipo local']
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Goles a favor visitante']=bbdd.at[y,'Goles a favor visitante']+bbdd.at[y,'Goles equipo visitante']
                break
            y=y-1

Rellenamos la columna de *Goles en contra local* y *Goles en contra visitante* a partir de los datos de las columnas *Goles equipo local* y *Goles equipo visitante* realizando una suma de los en contra que tienen en la jornada anterior más los goles en contra recibidos en la jornada anterior, teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'Goles en contra local'])) and (np.isnan(bbdd.loc[i,'Goles en contra visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Goles en contra local']=bbdd.at[j,'Goles en contra local']+bbdd.at[j,'Goles equipo visitante']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Goles en contra local']=bbdd.at[j,'Goles en contra visitante']+bbdd.at[j,'Goles equipo local']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Goles en contra visitante']=bbdd.at[j,'Goles en contra local']+bbdd.at[j,'Goles equipo visitante']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Goles en contra visitante']=bbdd.at[j,'Goles en contra visitante']+bbdd.at[j,'Goles equipo local']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'Goles en contra local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Goles en contra local']=bbdd.at[x,'Goles en contra local']+bbdd.at[x,'Goles equipo visitante']
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Goles en contra local']=bbdd.at[x,'Goles en contra visitante']+bbdd.at[x,'Goles equipo local']
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'Goles en contra visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Goles en contra visitante']=bbdd.at[y,'Goles en contra local']+bbdd.at[y,'Goles equipo visitante']
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Goles en contra visitante']=bbdd.at[y,'Goles en contra visitante']+bbdd.at[y,'Goles equipo local']
                break
            y=y-1

Rellenamos la columna de *Porterías a cero local* y *Porterías a cero visitante* a partir de los datos de las columnas *Goles equipo local* y *Goles equipo visitante* viendo si en la anterior jornada los goles en contra son igual a 0, que sería una portería a cero más, o no, teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'Porterías a cero local'])) and (np.isnan(bbdd.loc[i,'Porterías a cero visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Goles equipo visitante']==0:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero local']+1
                else:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero local']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Goles equipo local']==0:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero visitante']+1
                else:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero visitante']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Goles equipo visitante']==0:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero local']+1
                else:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero local']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Goles equipo local']==0:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero visitante']+1
                else:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero visitante']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'Porterías a cero local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                if bbdd.at[j,'Goles equipo visitante']==0:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero local']+1
                else:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero local']
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                if bbdd.at[j,'Goles equipo local']==0:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero visitante']+1
                else:
                    bbdd.at[i,'Porterías a cero local']=bbdd.at[j,'Porterías a cero visitante']
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'Porterías a cero visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[j,'Goles equipo visitante']==0:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero local']+1
                else:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero local']
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[j,'Goles equipo local']==0:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero visitante']+1
                else:
                    bbdd.at[i,'Porterías a cero visitante']=bbdd.at[j,'Porterías a cero visitante']
                break
            y=y-1

Rellenamos la columna de *Total tiros local* y *Total tiros visitante* a partir de los datos de las columnas *Disparos equipo local* y *Disparos equipo visitante* realizando una suma de los tiros que tienen en la jornada anterior más los tiros conseguidos en la jornada anterior, teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((np.isnan(bbdd.loc[i,'Total tiros local'])) and (np.isnan(bbdd.loc[i,'Total tiros visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Total tiros local']=bbdd.at[j,'Total tiros local']+bbdd.at[j,'Disparos equipo local']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                bbdd.at[i,'Total tiros local']=bbdd.at[j,'Total tiros visitante']+bbdd.at[j,'Disparos equipo visitante']
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Total tiros visitante']=bbdd.at[j,'Total tiros local']+bbdd.at[j,'Disparos equipo local']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                bbdd.at[i,'Total tiros visitante']=bbdd.at[j,'Total tiros visitante']+bbdd.at[j,'Disparos equipo visitante']
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (np.isnan(bbdd.loc[i,'Total tiros local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Total tiros local']=bbdd.at[x,'Total tiros local']+bbdd.at[x,'Disparos equipo local']
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                bbdd.at[i,'Total tiros local']=bbdd.at[x,'Total tiros visitante']+bbdd.at[x,'Disparos equipo visitante']
                break
            x=x-1
    elif (np.isnan(bbdd.loc[i,'Total tiros visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Total tiros visitante']=bbdd.at[y,'Total tiros local']+bbdd.at[y,'Disparos equipo local']
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                bbdd.at[i,'Total tiros visitante']=bbdd.at[y,'Total tiros visitante']+bbdd.at[y,'Disparos equipo visitante']
                break
            y=y-1

Rellenamos la columna de *Últimos partidos local* y *Últimos partidos visitante* a partir de los datos de las columnas *Puntos equipo local* y *Puntos equipo visitante* añadiendo al string una "V" si se han conseguido 3 puntos en la jornada anterior, una "E" si se ha conseguido 1 punto en la jornada anterior o una "D" se se han conseguido 0 puntos en la jornada anterior, teniendo en cuenta el equipo en cada caso.

In [ ]:
for i in range(0,10855):
    if((pd.isnull(bbdd.loc[i,'Últimos partidos local'])) and (pd.isnull(bbdd.loc[i,'Últimos partidos visitante']))):
        rellenado_local = False
        rellenado_visitante = False
        j=i-1
        while(j>=0):
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Puntos equipo local'] == 3:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos local'][1:] + "V"
                elif bbdd.at[j,'Puntos equipo local'] == 1:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos local'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos local'][1:] + "D"
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo local']) and(rellenado_local == False):
                if bbdd.at[j,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos visitante'][1:] + "V"
                elif bbdd.at[j,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos visitante'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[j,'Últimos partidos visitante'][1:] + "D"
                rellenado_local = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo local']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Puntos equipo local'] == 3:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos local'][1:] + "V"
                elif bbdd.at[j,'Puntos equipo local'] == 1:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos local'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos local'][1:] + "D"
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            if (bbdd.at[j,'Equipo visitante']==bbdd.at[i,'Equipo visitante']) and(rellenado_visitante == False):
                if bbdd.at[j,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos visitante'][1:] + "V"
                elif bbdd.at[j,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos visitante'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[j,'Últimos partidos visitante'][1:] + "D"
                rellenado_visitante = True
                if rellenado_local and rellenado_visitante:
                    rellenado_local = False
                    rellenado_visitante = False
                    break
            j=j-1
    elif (pd.isnull(bbdd.loc[i,'Últimos partidos local'])):
        x=i-1
        while(x>=0):
            if bbdd.at[x,'Equipo local']==bbdd.at[i,'Equipo local']:
                if bbdd.at[x,'Puntos equipo local'] == 3:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos local'][1:] + "V"
                elif bbdd.at[x,'Puntos equipo local'] == 1:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos local'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos local'][1:] + "D"
                break
            if bbdd.at[x,'Equipo visitante']==bbdd.at[i,'Equipo local']:
                if bbdd.at[x,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos visitante'][1:] + "V"
                elif bbdd.at[x,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos visitante'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos local']=bbdd.at[x,'Últimos partidos visitante'][1:] + "D"
                break
            x=x-1
    elif (pd.isnull(bbdd.loc[i,'Últimos partidos visitante'])):
        y=i-1
        while(y>=0):
            if bbdd.at[y,'Equipo local']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[y,'Puntos equipo local'] == 3:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos local'][1:] + "V"
                elif bbdd.at[y,'Puntos equipo local'] == 1:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos local'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos local'][1:] + "D"
                break
            if bbdd.at[y,'Equipo visitante']==bbdd.at[i,'Equipo visitante']:
                if bbdd.at[y,'Puntos equipo visitante'] == 3:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos visitante'][1:] + "V"
                elif bbdd.at[y,'Puntos equipo visitante'] == 1:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos visitante'][1:] + "E"
                else:
                    bbdd.at[i,'Últimos partidos visitante']=bbdd.at[y,'Últimos partidos visitante'][1:] + "D"
                break
            y=y-1

## Descripción y exploración de los datos

Creamos el informe usando la librería de pandas_profiling y el método *ProfileReport* con el dataframe que hemos obtenido con el apartado de recolección de los datos.

In [ ]:
profile_report = pandas_profiling.ProfileReport(bbdd)

Mostramos el informe para ver los resultados de los datos.

In [ ]:
profile_report.to_widgets()

Mostramos un primer gráfico para ver la probabilidad de cada clase de la variable objetivo.

In [ ]:
plt.pie([(bbdd['Resultado'] == 0).sum(),(bbdd['Resultado'] == 1).sum(),(bbdd['Resultado'] == 2).sum()],labels=["Local","Empate","Visitante"],autopct="%0.01f %%")
plt.axis("equal")
plt.show()

### Variables numéricas

Realizamos un gráfico para mostrar la distribución de las variables numéricas de nuestra base de datos.

In [ ]:
fig, axes = plt.subplots(nrows=19, ncols=2, figsize=(15, 60))
axes = axes.flat
columnas_numeric = bbdd.select_dtypes(include=['float64', 'int64']).columns
columnas_numeric = columnas_numeric.drop('Resultado')

for i, colum in enumerate(columnas_numeric):
    sns.histplot(
        data    = bbdd,
        x       = colum,
        stat    = "count",
        kde     = True,
        line_kws= {'linewidth': 2},
        alpha   = 0.3,
        ax      = axes[i]
    )
    axes[i].set_title(colum, fontweight = "bold")
    axes[i].tick_params()
    axes[i].set_xlabel("")

for i in [37]:
    fig.delaxes(axes[i])
    
fig.tight_layout()
plt.subplots_adjust(top = 0.95)
fig.suptitle('Distribución variables numéricas', fontweight = "bold")
sfig = fig.get_figure()
sfig.savefig('Distribución_variables_numéricas.jpg')

Mostramos otro gráfico para ver la correlación de las variables predictoras con respecto a la variable objetivo.

In [ ]:
fig, axes = plt.subplots(nrows=19, ncols=2, figsize=(15, 60))
axes = axes.flat
columnas_numeric = bbdd.select_dtypes(include=['float64', 'int64']).columns
columnas_numeric = columnas_numeric.drop('Resultado')

for i, colum in enumerate(columnas_numeric):
    sns.regplot(
        x           = bbdd[colum],
        y           = bbdd['Resultado'],
        color       = "gray",
        marker      = '.',
        scatter_kws = {"alpha":0.4},
        line_kws    = {"color":"r","alpha":0.7},
        ax          = axes[i]
    )
    axes[i].set_title(f"Resultado vs {colum}", fontweight = "bold")
    axes[i].yaxis.set_major_formatter(ticker.EngFormatter())
    axes[i].xaxis.set_major_formatter(ticker.EngFormatter())
    axes[i].tick_params()
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    
for i in [37]:
    fig.delaxes(axes[i])
    
plt.subplots_adjust(top=0.95)
fig.suptitle('Correlación con resultado variables numéricas', fontweight = "bold")
sfig = fig.get_figure()
sfig.savefig('Correlación_con_resultado_variables_numéricas.jpg')

Ahora sacamos gráficas de cada variable con respecto a la variable objetivo, usando cuadrículas para múltiples gráficas (sns.FacetGrid), histogramas (px.histogram) y matrices de dispersión para tenerlas en cuenta más adelante en el desarrollo de la preparación de los datos a la hora de la discretización de variables numéricas.

In [ ]:
g = sns.FacetGrid(bbdd, col="Año")
g = g.map(plt.hist, "Resultado")

In [ ]:
px.histogram(bbdd, x='Jornada', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Jornada"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Puntos totales local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Puntos totales visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Puntos totales local","Puntos totales visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='PG local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='PG visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["PG local","PG visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='PE local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='PE visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["PE local","PE visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='PP local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='PP visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["PP local","PP visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles a favor local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles a favor visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Goles a favor local","Goles a favor visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles en contra local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles en contra visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Goles en contra local","Goles en contra visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Porterías a cero local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Porterías a cero visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Porterías a cero local","Porterías a cero visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Total tiros local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Total tiros visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Total tiros local","Total tiros visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Goles equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Goles equipo local","Goles equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Diferencia de goles', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Diferencia de goles"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Posesión equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Posesión equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Posesión equipo local","Posesión equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Disparos equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Disparos equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Disparos equipo local","Disparos equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Disparos a puerta equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Disparos a puerta equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Disparos a puerta equipo local","Disparos a puerta equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Tarjetas amarillas equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Tarjetas amarillas equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Tarjetas amarillas equipo local","Tarjetas amarillas equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Tarjetas rojas equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Tarjetas rojas equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Tarjetas rojas equipo local","Tarjetas rojas equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Faltas equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Faltas equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Faltas equipo local","Faltas equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Córners equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Córners equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Córners equipo local","Córners equipo visitante"], color='Resultado')

In [ ]:
px.histogram(bbdd, x='Paradas equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Paradas equipo visitante', color='Resultado')

In [ ]:
px.scatter_matrix(bbdd, dimensions=["Paradas equipo local","Paradas equipo visitante"], color='Resultado')

### Variables categóricas

Mostramos un gráfico para ver la correlación existente entre las variables categóricas y discretas respecto a la variable objetivo, al igual que hicimos con las variables numéricas.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(30, 20))
axes = axes.flat
columnas_object = bbdd.select_dtypes(include=['object']).columns

for i, colum in enumerate(columnas_object):
    sns.violinplot(
        x     = colum,
        y     = 'Resultado',
        data  = bbdd,
        color = "white",
        ax    = axes[i]
    )
    axes[i].set_title(f"Resultado vs {colum}",fontweight = "bold")
    axes[i].yaxis.set_major_formatter(ticker.EngFormatter())
    axes[i].tick_params()
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    
fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Correlación con resultado variables discretas', fontweight = "bold");
sfig = fig.get_figure()
sfig.savefig('Correlación_con_resultado_variables_discretas.jpg')

De nuevo, sacamos gráficas de cada variable con respecto a la variable objetivo, usando cuadrículas para múltiples gráficas (sns.FacetGrid) e histogramas (px.histogram).

In [ ]:
px.histogram(bbdd, x='Fecha', color='Resultado')

In [ ]:
g = sns.FacetGrid(bbdd, col="País")
g = g.map(plt.hist, "Resultado")

In [ ]:
px.histogram(bbdd, x='Equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Equipo visitante', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Últimos partidos local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Últimos partidos visitante', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Marcador', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Puntos equipo local', color='Resultado')

In [ ]:
px.histogram(bbdd, x='Puntos equipo visitante', color='Resultado')

Guardamos la base de datos en formato .csv para exportarla y poderla utilizar en el siguiente notebook de aprendizaje de modelos de Machine Learning.

In [ ]:
bbdd.to_csv('bbdd.csv',encoding='utf-8')